In [1]:
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt

--2025-02-07 07:11:10--  https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26622 (26K) [text/plain]
Saving to: ‘edgar_allan_poe.txt’

edgar_allan_poe.txt 100%[===================>]  26.00K  --.-KB/s    in 0.002s  

2025-02-07 07:11:10 (16.2 MB/s) - ‘edgar_allan_poe.txt’ saved [26622/26622]

--2025-02-07 07:11:10--  https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP re

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.model_selection import train_test_split

In [3]:
input_files = [ 'edgar_allan_poe.txt' , 'robert_frost.txt',]

In [4]:
!head edgar_allan_poe.txt

LO! Death hath rear'd himself a throne
In a strange city, all alone,
Far down within the dim west
Where the good, and the bad, and the worst, and the best,
Have gone to their eternal rest.
 
There shrines, and palaces, and towers
Are not like any thing of ours
Oh no! O no! ours never loom
To heaven with that ungodly gloom!


In [5]:
!head robert_frost.txt

Two roads diverged in a yellow wood,
And sorry I could not travel both
And be one traveler, long I stood
And looked down one as far as I could
To where it bent in the undergrowth; 

Then took the other, as just as fair,
And having perhaps the better claim
Because it was grassy and wanted wear,
Though as for that the passing there


In [6]:
#data into lists
input_texts = []
labels = []

for label,f in enumerate(input_files):
  print(f"{f} corresponds to {label}")
  for line in open(f):
    line = line.rstrip().lower()
    if line:
      line = line.translate(str.maketrans('','',string.punctuation))

      input_texts.append(line)
      labels.append(label)

edgar_allan_poe.txt corresponds to 0
robert_frost.txt corresponds to 1


In [7]:
train_text, test_text, Ytrain, Ytest = train_test_split(input_texts, labels)

In [8]:
len(Ytrain), len(Ytest)

(1615, 539)

In [9]:
train_text[:5]

['not to sink under being man and wife',
 'this is the pit from which we starks were digged',
 'thy grace thy more than beauty',
 'once more before we meet elsewhere',
 'i was a baby i dont know where i was']

In [10]:
Ytrain[:5]

[1, 1, 0, 1, 1]

In [11]:
idx =1
word2idx = {'<unk>':0}

In [12]:
#populate word2idx
for text in train_text:
  tokens = text.split()
  for token in tokens:
    if token not in word2idx:
      word2idx[token] = idx
      idx+=1

In [13]:
word2idx

{'<unk>': 0,
 'not': 1,
 'to': 2,
 'sink': 3,
 'under': 4,
 'being': 5,
 'man': 6,
 'and': 7,
 'wife': 8,
 'this': 9,
 'is': 10,
 'the': 11,
 'pit': 12,
 'from': 13,
 'which': 14,
 'we': 15,
 'starks': 16,
 'were': 17,
 'digged': 18,
 'thy': 19,
 'grace': 20,
 'more': 21,
 'than': 22,
 'beauty': 23,
 'once': 24,
 'before': 25,
 'meet': 26,
 'elsewhere': 27,
 'i': 28,
 'was': 29,
 'a': 30,
 'baby': 31,
 'dont': 32,
 'know': 33,
 'where': 34,
 'began': 35,
 'stairs': 36,
 'two': 37,
 'footsteps': 38,
 'for': 39,
 'each': 40,
 'step': 41,
 'leaves': 42,
 'they': 43,
 'withering': 44,
 'sere': 45,
 'live': 46,
 'together': 47,
 'then': 48,
 'pull': 49,
 'apart': 50,
 'in': 51,
 'fever': 52,
 'of': 53,
 'minute': 54,
 'but': 55,
 'dimremembered': 56,
 'story': 57,
 'end': 58,
 'far': 59,
 'my': 60,
 'going': 61,
 'forth': 62,
 'stranger': 63,
 'cousins': 64,
 'family': 65,
 'tree': 66,
 'thus': 67,
 'pacified': 68,
 'psyche': 69,
 'kissed': 70,
 'her': 71,
 'no': 72,
 'alas': 73,
 'that': 7

In [14]:
len(word2idx)

2513

In [15]:
#data into integer format
train_text_int = []
test_text_int = []

for text in train_text:
  tokens = text.split()
  line_as_int = [word2idx[token] for token in tokens]
  train_text_int.append(line_as_int)

for text in test_text:
  tokens = text.split()
  line_as_int = [word2idx.get(token,0) for token in tokens]
  test_text_int.append(line_as_int)

In [16]:
train_text_int[100:105]

[[53, 60, 391, 392, 393],
 [189, 394, 11, 395, 7, 396, 115],
 [397, 28, 226, 223, 398, 127, 10, 241, 302, 399],
 [259, 164, 28, 400, 401, 91, 2, 11, 402],
 [53, 11, 173, 403, 397, 404, 10, 11, 405]]

In [17]:
#initialising A and pi matrices - for both classes
V = len(word2idx)

A0 = np.ones((V,V))
pi0 = np.ones(V)

A1 = np.ones((V,V))
pi1 = np.ones(V)

In [18]:
#Computing counts for A and pi
def compute_counts(text_as_int, A, pi):
  for tokens in text_as_int:
    last_idx = None
    for idx in tokens:
      if last_idx is None:
        pi[idx] +=1
      else:
        A[last_idx,idx] +=1

      last_idx = idx

  compute_counts([t for t, y in zip(train_text_int, Ytrain) if y==0], A0, pi0)
  compute_counts([t for t, y in zip(train_text_int, Ytrain) if y==1], A1, pi1)

In [19]:
#Normalising A and pi
A0 /= A0.sum(axis=1, keepdims=True)
pi0 /= pi0.sum()

A1 /= A1.sum(axis=1, keepdims=True)
pi1 /= pi1.sum()

In [20]:
logA0 = np.log(A0)
logpi0 = np.log(pi0)

logA1 = np.log(A1)
logpi1 = np.log(pi1)

In [21]:
count0 = sum(y==0 for y in Ytrain)
count1 = sum(y==1 for y in Ytrain)
total = len(Ytrain)
p0 = count0 / total
p1 = count1 / total
logp0 = np.log(p0)
logp1 = np.log(p1)
p0, p1

(0.3368421052631579, 0.6631578947368421)

In [22]:
#Building a Classifier
class Classifier:
    def __init__(self, logAs, logpis, logpriors):
        self.logAs = logAs
        self.logpis = logpis
        self.logpriors = logpriors
        self.K = len(logpriors)  # number of classes


    def _compute_log_likelihood(self, input_, class_):
      logA = self.logAs[class_]
      logpi = self.logpis[class_]

      last_idx = None
      logprob = 0
      for idx in input_:
        if last_idx is None:
          logprob += logpi[idx]
        else:
          logprob += logA[last_idx, idx]

        last_idx = idx

      return logprob

    def predict(self, inputs):
      predictions = np.zeros(len(inputs))
      for i, input_ in enumerate(inputs):
        posteriors = [self._compute_log_likelihood(input_, c) + self.logpriors[c] \
                      for c in range(self.K)]
        pred = np.argmax(posteriors)
        predictions[i] = pred
      return predictions


In [23]:
clf = Classifier([logA0, logA1], [logpi0, logpi1], [logp0, logp1])

In [24]:
Ptrain = clf.predict(train_text_int)
print(f"Train acc: {np.mean(Ptrain == Ytrain)}")

Train acc: 0.6631578947368421


In [25]:
Ptest = clf.predict(test_text_int)
print(f"Test acc: {np.mean(Ptest == Ytest)}")

Test acc: 0.6771799628942486


In [26]:
from sklearn.metrics import confusion_matrix, f1_score

In [27]:
cm = confusion_matrix(Ytrain, Ptrain)
cm

array([[   0,  544],
       [   0, 1071]])

In [28]:
cm_test = confusion_matrix(Ytest, Ptest)
cm_test

array([[  0, 174],
       [  0, 365]])

In [29]:
f1_score(Ytrain, Ptrain)

0.7974683544303798

In [30]:
f1_score(Ytest, Ptest)

0.8075221238938053